## 0. Set Args

In [1]:
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser

parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

args.data_dir = '/root/data/'
args.seed = 123
args.lr = 0.01

## 1. Off the shelf implementation

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
class LogisticRegressionLazy(nn.Module):

    def __init__(self, nx):
        super(LogisticRegressionLazy, self).__init__()
        self.scorer = nn.Linear(nx, 1)

    def forward(self, X):
        '''
        X has shape (m, nx)
        '''
        # shape (m, 1)
        z = self.scorer(X)
        # shape (m, 1)
        a = torch.sigmoid(z)
        return z, a

## 2. With custom linear module and sigmoid function

In [4]:
# extending pytorch (demo of custom function with custom forward backward, custom LinearFunction)
# https://pytorch.org/docs/master/notes/extending.html
# https://github.com/pytorch/pytorch/blob/c9bb990707d4bfe524f3f1c4a77ff85fed1cd2a2/torch/csrc/api/include/torch/nn/functional/loss.h

# pytorch Autograd function (RELU example)
# https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html

# discussion custom threshold forward and backward
# https://discuss.pytorch.org/t/how-to-call-only-backward-path-of-pytorch-function/22839/2

# Define custom autograd.Function and put the function in nn.Module
# https://discuss.pytorch.org/t/how-to-call-the-backward-function-of-a-custom-module/7853

class LogisticRegressionCustom(nn.Module):
    '''Linear and sigmoid with custom backward'''
    def __init__(self, nx, init_weight, init_bias):
        super(LogisticRegressionCustom, self).__init__()
        self.scorer = CustomLinearLayer(nx, init_weight, init_bias)
        self.sigmoid = CustomSigmoidFunction.apply
        
    def forward(self, X):
        '''
        X has shape (m, nx)
        '''
        # shape(m, ny=1)
        z = self.scorer(X)
        # shape(m, ny=1)
        a = self.sigmoid(z)
        return z, a  

class CustomSigmoidFunction(torch.autograd.Function):
    '''
    doesn't get backprop through because loss function takes in logit directly
    '''
    
    @staticmethod
    def forward(ctx, inp):
        '''
        inp: shape(m, ny)
        '''
        ctx.save_for_backward(inp)
        return 1 / (1 + torch.exp(-inp))
        
    @staticmethod
    def backward(ctx, dA):
        '''
        Demonstration purpose. Not used in overall backprop since our loss function computes with logits.
        dA: shape(m, ny)
        '''
        # retrieve cache
        inp, = ctx.saved_tensors
        grad_inp = None
        
        A = 1.0 / (1.0 + torch.exp(-inp))
        # shape(m, ny)
        grad_inp = A * (1 - A) * dA
        
        return grad_inp

class CustomLinearFunction(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, inp, wt, b):
        '''
        inp: shape(nx, m)
        wt: shape(ny=1, nx)
        b: shape(ny=1, 1)
        '''
        ctx.save_for_backward(inp, wt, b)
        # (ny, m) = (ny, nx)(nx, m) + (ny, 1)t
        z = wt.mm(inp) + b
        assert z.shape == (1, inp.shape[1])
        # (ny, m)
        return z
        
    @staticmethod
    def backward(ctx, dZ):
        '''
        dZ: shape(ny, m)
        '''
        
        # retrieve cache
        inp, wt, b = ctx.saved_tensors
        m = inp.shape[1]
        grad_inp, grad_wt, grad_b = None, None, None
        
        # Z = W dot X.T + b 
        # shape(nx, m)
        grad_inp = wt.t().mm(dZ)
        # shape(ny=1, nx)
        grad_wt = dZ.mm(inp.t())
        # shape(ny=1, 1)
        grad_b = torch.sum(dZ, dim=1, keepdim=True)
        
        return grad_inp, grad_wt, grad_b

    
class CustomLinearLayer(nn.Module):
    '''Linear with custom backward'''
    def __init__(self, nx, init_weight, init_bias):
        super(CustomLinearLayer, self).__init__()
        # init weight and bias
        self.weight = nn.Parameter(torch.tensor(init_weight))
        self.bias = nn.Parameter(torch.tensor(init_bias))
        
    def forward(self, X):
        '''
        X has shape (m, nx)
        '''
        # (m, ny=1)
        z = CustomLinearFunction.apply(X.t(), self.weight, self.bias).t()
        return z      

In [5]:
# gradient check Sigmoid
inp_test = torch.rand(10, 1, requires_grad=True).double()
assert torch.autograd.gradcheck(CustomSigmoidFunction.apply, (inp_test,), raise_exception=True)

In [6]:
# gradient check CustomLinear
inp_test = torch.rand(5, 1000, requires_grad=True).double()
wt_test = torch.rand(1, 5,requires_grad=True).double()
b_test = torch.rand(1, 1,requires_grad=True).double()
assert torch.autograd.gradcheck(CustomLinearFunction.apply, (inp_test, wt_test, b_test), raise_exception=True)

## 3. With custom loss function

In [7]:
# stable loss implementation
# tensorflow demo
# https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits
# Pytorch source code
# https://github.com/pytorch/pytorch/blob/7d6d5f4be0da26079bc81ca49265cde713a75051/aten/src/ATen/native/Loss.cpp#L201

# how to write a pytorch loss autograd.function with backward vs nn.module with only forward
# https://discuss.pytorch.org/t/custom-loss-autograd-module-what-is-the-difference/69251

# DeepLearning Specialization Homework
# https://github.com/Chucooleg/DeepLearning_Specialization_Assignments/blob/master/course%201%20Assignments/Week%202/Logistic%20Regression%20as%20a%20Neural%20Network/Logistic_Regression_with_a_Neural_Network_mindset_v6a.ipynb

class CustomBCEWithLogitLoss(torch.autograd.Function):
    '''
    Custom Binary Cross Entropy Loss with Logits.
    Implementation Goal -- Numerically stable implementation.
    '''
    
    @staticmethod
    def forward(ctx, Z, Y):
        '''
        Z: Pre-Activations(i.e. Logits), shape(m, ny=1)
        Y: Predictions, shape(m, ny=1)
        '''
        ctx.save_for_backward(Z, Y)
        
        # this intuitive version is not numerically stable if Z is a large -ve number
#         A = 1 / (1 + torch.exp(-Z))
#         loss = - torch.mean(Y * torch.log(A) + (1 - Y) * torch.log(1 - A))
        
        # follow this tensorflow implmentation
        # https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits
        loss = torch.max(Z, torch.zeros(Z.shape, dtype=Z.dtype)) - Z * Y + torch.log(1 + torch.exp(-torch.abs(Z)))
        loss = torch.mean(loss)

        return loss
    
    @staticmethod
    def backward(ctx, grad_output):
 
        # retrieve cache
        Z, Y = ctx.saved_tensors
        grad_Z, grad_Y = None, None
        m = Z.shape[0]
        
        # https://github.com/pytorch/pytorch/blob/7d6d5f4be0da26079bc81ca49265cde713a75051/aten/src/ATen/native/Loss.cpp#L226
        grad_Z = (torch.sigmoid(Z) - Y) * grad_output / m
        grad_Y = - Z * grad_output / m
        
        return grad_Z, grad_Y

In [8]:
# Gradcheck a custom loss function
# https://discuss.pytorch.org/t/how-to-check-the-gradients-of-custom-implemented-loss-function/8546

# gradient check CustomBCEWithLogitLoss
Z_test = torch.rand(10, 1,requires_grad=True).double()
Y_test = torch.rand(10, 1,requires_grad=True).double()
assert torch.autograd.gradcheck(CustomBCEWithLogitLoss.apply, (Z_test, Y_test), raise_exception=True)

## 3. With custom optimizer

In [9]:
# Custom Optimizer Tutorial
# http://mcneela.github.io/machine_learning/2019/09/03/Writing-Your-Own-Optimizers-In-Pytorch.html
# https://huggingface.co/transformers/_modules/transformers/optimization.html#AdamW

from torch.optim import Optimizer

class CustomSGD(Optimizer):
    
    def __init__(self, params, lr=1e-3):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {} - should be >= 0.0".format(lr))
        defaults = dict(lr=lr)
        super(CustomSGD, self).__init__(params, defaults)
        
    def step(self, closure=None):
        '''performs single optimization step'''
        loss = None
        
        for group in self.param_groups:
            for p in group['params']:
                
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError("Adam does not support sparse gradients, please consider SparseAdam instead")
                
                p.data.add_(grad, alpha=-group['lr'])
        
        return loss     

## 3. Main Train & Pred Loop

In [10]:
def train(model, train_loader, valid_loader, loss_criterion, optimizer, args, epochs=20):
    '''
    Train model and report losses on train and dev sets per epoch
    '''
    
    history = {
        'train_losses': [],
        'valid_losses': [],        
        'valid_accuracy': [],
        'weights': [],
        'bias': [],
    }

    # save parameters
    write_param_history(model, history)
    
    for epoch_i in range(epochs):

        # train
        model.train()
        batch_losses = []
        for batch_i, batch_data in enumerate(train_loader):
            logits, activations = model(batch_data['X'])
            loss = loss_criterion(logits, batch_data['y'])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            batch_losses.append(loss.item())
        history['train_losses'].append(sum(batch_losses) / len(batch_losses))

        # validate
        batch_val_losses, batch_val_accuracies = pred(model, valid_loader, loss_criterion)
        history['valid_losses'].append(sum(batch_val_losses) / len(batch_val_losses))
        history['valid_accuracy'].append(sum(batch_val_accuracies) / len(batch_val_accuracies))

        # save parameters
        write_param_history(model, history)
        
    return history

def write_param_history(model, history):
    weights = model.scorer.weight.clone().detach().numpy()
    bias = model.scorer.bias.data.clone().detach().numpy()
    history['weights'].append(weights)
    history['bias'].append(bias)    

In [11]:
@torch.no_grad()
def pred(model, test_loader, loss_criterion):
    '''Propogate forward on dev or test set, report loss and accuracy.'''
    
    # evaluate
    model.eval()
    batch_losses = []
    batch_accuracies = []
    for batch_i, batch_data in enumerate(test_loader):
        logits, activations = model(batch_data['X'])
        loss = loss_criterion(logits, batch_data['y'])
        batch_losses.append(loss.item())
        accuracy = torch.mean((activations > 0.5).type(torch.FloatTensor).eq(batch_data['y']).type(torch.FloatTensor))
        batch_accuracies.append(accuracy.item())
    
    return batch_losses, batch_accuracies

## 4. Make Toy Dataset

In [12]:
# Pytorch Dataloader
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

# Pytorch Data Collate (Further reading, not implemented here)
# https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader

import os
import numpy as np
from sklearn.datasets import make_classification
from torch.utils.data import Dataset, DataLoader

class ToyDataset(Dataset):
    """Toy dataset for Logistic Regression."""

    def __init__(self, data_dir):
        """
        Args:
            data_dir (string): Path to the directory with data files.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        # shape (m, nx)
        self.X = np.load(os.path.join(data_dir, 'features.npy'))
        # shape (m, ny=1)
        self.y = np.load(os.path.join(data_dir, 'labels.npy'))

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        X = torch.from_numpy(self.X[idx, :]).type(torch.FloatTensor)
        y = torch.from_numpy(self.y[idx, :]).type(torch.FloatTensor)
        sample = {'X': X, 'y': y}

        return sample

In [13]:
# construct and save toydataset

m_train, m_valid, m_test = 90, 500, 500
m_total = m_train + m_valid + m_test

X, y = make_classification(n_samples=m_total, n_features=10, n_informative=10, n_redundant=0, n_repeated=0, n_classes=1, n_clusters_per_class=4, weights=None, flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=args.seed)
y = np.expand_dims(y, -1)

np.random.seed(123)
permutation = np.random.permutation(m_total)
print('First 10 training indices', permutation[:10])
print('X shape', X.shape)
print('y shape', y.shape)

train_indices = permutation[0:m_train]
valid_indices = permutation[m_train:m_train+m_valid]
test_indices = permutation[m_train+m_valid:m_test]

np.save(os.path.join(args.data_dir, 'toy_lr_1', 'train', 'features.npy'), X[train_indices])
np.save(os.path.join(args.data_dir, 'toy_lr_1', 'train', 'labels.npy'), y[train_indices])

np.save(os.path.join(args.data_dir, 'toy_lr_1', 'valid', 'features.npy'), X[valid_indices])
np.save(os.path.join(args.data_dir, 'toy_lr_1', 'valid', 'labels.npy'), y[valid_indices])

np.save(os.path.join(args.data_dir, 'toy_lr_1', 'test', 'features.npy'), X[test_indices])
np.save(os.path.join(args.data_dir, 'toy_lr_1', 'test', 'labels.npy'), y[test_indices])

First 10 training indices [1037  655  547  487  307  689  856  309  260  229]
X shape (1090, 10)
y shape (1090, 1)


## 5. Train and compare results on toy dataset

In [39]:
batch_size = 5

training_set = ToyDataset(data_dir=os.path.join(args.data_dir, 'toy_lr_1', 'train'))
training_generator = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True)

validation_set = ToyDataset(data_dir=os.path.join(args.data_dir, 'toy_lr_1', 'valid'))
validation_generator = torch.utils.data.DataLoader(validation_set, batch_size=batch_size)

test_set = ToyDataset(data_dir=os.path.join(args.data_dir, 'toy_lr_1', 'test'))
test_generator = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

m = training_set.X.shape[0]
nx = training_set.X.shape[1]
ny = 1

In [41]:
torch.manual_seed(args.seed)

# set off-the-shelf model, loss function and optimizer
model = LogisticRegressionLazy(nx)
loss_criterion_lazy = nn.BCEWithLogitsLoss(reduction='mean')
optimizer_lazy = torch.optim.SGD(model.parameters(), lr=args.lr)

history_off_the_shelf = train(model, training_generator, validation_generator, loss_criterion, optimizer, args, epochs=10)

In [46]:
history_off_the_shelf['weights'][0]

array([[-0.12895013,  0.01047492, -0.15705723,  0.11925378, -0.26944348,
         0.23180881, -0.22984707, -0.25141433, -0.19982024,  0.1432175 ]],
      dtype=float32)

In [47]:
history_off_the_shelf['bias'][0]

array([-0.11684369], dtype=float32)

In [52]:
torch.manual_seed(args.seed)

wt_arr = [[-0.12895013,  0.01047492, -0.15705723,  0.11925378, -0.26944348,
         0.23180881, -0.22984707, -0.25141433, -0.19982024,  0.1432175 ]]
bias_arr = [[-0.11684369]]

# set custom model, loss function and optimizer
model = LogisticRegressionCustom(nx, init_weight=wt_arr, init_bias=bias_arr)
loss_criterion = CustomBCEWithLogitLoss.apply
optimizer = CustomSGD(model.parameters(), lr=args.lr)

history_custom = train(model, training_generator, validation_generator, loss_criterion, optimizer, args, epochs=10)

### 5.1 Cross-comparison on train & valid loss, accuracy and parameter values

In [56]:
list(zip(history_custom['train_losses'], history_off_the_shelf['train_losses']))

[(0.6196715947654512, 0.6201359927654266),
 (0.46291036903858185, 0.4648684403962559),
 (0.3668956657250722, 0.36838793257872265),
 (0.3055238318112161, 0.3070671570797761),
 (0.2641151857872804, 0.2640540964073605),
 (0.2331896329091655, 0.23380550162659752),
 (0.2104352960983912, 0.21100338507029745),
 (0.19241697672340605, 0.1930240887320704),
 (0.17750125378370285, 0.17796090824736488),
 (0.16582258914907774, 0.16588551302750906)]

In [57]:
list(zip(history_custom['valid_losses'], history_off_the_shelf['valid_losses']))

[(0.5399632832407951, 0.5413854518532752),
 (0.435880730599165, 0.4370486372709274),
 (0.37145381346344947, 0.37267079040408135),
 (0.32947849318385125, 0.3301878882944584),
 (0.3007757315039635, 0.30113270990550517),
 (0.27999262131750585, 0.28022984832525255),
 (0.264319948926568, 0.26447768017649653),
 (0.2520730609446764, 0.25213010914623735),
 (0.24232644483447074, 0.24237974375486374),
 (0.23437572184950115, 0.2343392314016819)]

In [59]:
list(zip(history_custom['valid_accuracy'], history_off_the_shelf['valid_accuracy']))

[(0.718000012487173, 0.718000012487173),
 (0.7900000104308128, 0.7900000104308128),
 (0.8500000083446503, 0.8480000084638596),
 (0.8820000067353249, 0.8820000067353249),
 (0.8980000054836274, 0.8980000054836274),
 (0.9120000049471855, 0.9140000048279763),
 (0.9240000042319297, 0.9240000042319297),
 (0.9320000037550926, 0.9320000037550926),
 (0.9340000036358833, 0.9340000036358833),
 (0.9320000037550926, 0.9320000037550926)]

In [60]:
list(zip(history_custom['weights'], history_off_the_shelf['weights']))

[(array([[-0.12895013,  0.01047492, -0.15705723,  0.11925378, -0.26944348,
           0.23180881, -0.22984707, -0.25141433, -0.19982024,  0.1432175 ]],
        dtype=float32),
  array([[-0.12895013,  0.01047492, -0.15705723,  0.11925378, -0.26944348,
           0.23180881, -0.22984707, -0.25141433, -0.19982024,  0.1432175 ]],
        dtype=float32)),
 (array([[-0.08323156, -0.04968001, -0.2359461 ,  0.07982081, -0.26038086,
           0.29659656, -0.178131  , -0.18310717, -0.15923896,  0.08490142]],
        dtype=float32),
  array([[-0.08386043, -0.0485882 , -0.23378837,  0.07998081, -0.2593329 ,
           0.29605865, -0.17820223, -0.18384506, -0.15970805,  0.08506422]],
        dtype=float32)),
 (array([[-0.04662747, -0.09852936, -0.29087773,  0.04752795, -0.24887085,
           0.34518456, -0.13998047, -0.13147025, -0.12872545,  0.03860396]],
        dtype=float32),
  array([[-0.04731767, -0.09730133, -0.28897136,  0.04735611, -0.24824813,
           0.3449241 , -0.13994777, -0.1325

In [146]:
list(zip(history_custom['bias'], history_off_the_shelf['bias']))

[(array([[-0.11684369]], dtype=float32), array([-0.11684369], dtype=float32)),
 (array([[-0.19164065]], dtype=float32), array([-0.1915197], dtype=float32)),
 (array([[-0.25272644]], dtype=float32), array([-0.25295103], dtype=float32)),
 (array([[-0.30405846]], dtype=float32), array([-0.30448583], dtype=float32)),
 (array([[-0.3483866]], dtype=float32), array([-0.34896126], dtype=float32)),
 (array([[-0.38755324]], dtype=float32), array([-0.38814685], dtype=float32)),
 (array([[-0.42270768]], dtype=float32), array([-0.42335588], dtype=float32)),
 (array([[-0.45478928]], dtype=float32), array([-0.4554978], dtype=float32)),
 (array([[-0.48439842]], dtype=float32), array([-0.4851668], dtype=float32)),
 (array([[-0.5119091]], dtype=float32), array([-0.51271677], dtype=float32)),
 (array([[-0.5377259]], dtype=float32), array([-0.5385727], dtype=float32))]

## 6. Further reading

In [ ]:
# Pytorch Data Collate (Further reading, not implemented here)
# https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader

# Karpathy resource on a tiny implementation of  autodiff from scratch if anyone is interested. Engine.py is where the meat is
# https://github.com/karpathy/micrograd/blob/master/micrograd/engine.py